In [1]:
!pip install sentence-transformers
!pip install pyarrow

In [2]:
import pandas as pd
import sentence_transformers as st
import os
import boto3
from io import BytesIO
import pyarrow.parquet as pq

In [3]:
# Download from the 🤗 Hub
model = st.SentenceTransformer("pablosi/bge-m3-trained-2")

In [4]:
# Initialize S3 client
s3_client = boto3.client('s3')

# Define your bucket and folder path
bucket_name = 'boe-facil'
folder_path = 'parquet_chunks'

# List all parquet files in the S3 folder
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)

parquet_files = []
if 'Contents' in response:
    for obj in response['Contents']:
        if obj['Key'].endswith('.parquet'):
            parquet_files.append(obj['Key'])

print(f"{len(parquet_files)} parquet filesfound")

# Read each parquet file one by one
for file_key in parquet_files[:1]:
    print(f"Reading file: {file_key}")

    # Get the object from S3
    obj = s3_client.get_object(Bucket=bucket_name, Key=file_key)

    # Read parquet file into pandas dataframe
    df = pd.read_parquet(BytesIO(obj['Body'].read()))

626 parquet filesfound
Reading file: parquet_chunks/boe_data_20230102.parquet


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 374 entries, 0 to 373
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   sumario_id           374 non-null    object
 1   seccion_codigo       374 non-null    object
 2   seccion_nombre       374 non-null    object
 3   departamento_codigo  374 non-null    object
 4   departamento_nombre  374 non-null    object
 5   epigrafe_nombre      374 non-null    object
 6   item_id              374 non-null    object
 7   item_titulo          374 non-null    object
 8   chunk_id             374 non-null    object
 9   chunk_numero         374 non-null    int64 
 10  fecha_publicacion    374 non-null    object
 11  diario_numero        374 non-null    object
 12  publicacion          374 non-null    object
 13  texto                374 non-null    object
 14  tokens_aproximados   374 non-null    int64 
 15  total_chunks_fila    374 non-null    int64 
 16  sumario_url_p

In [ ]:
embeddings = model.encode_document(
    sentences=df["texto"],
    output_value="sentence_embedding",
    show_progress_bar=True,
)
print(embeddings.shape)
df["embeddings"] = embeddings.tolist()

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
